In [ ]:
import pickle

train_mat=pickle.load(open("train_mat.file", "rb" ) )
val_mat=pickle.load(open("val_mat.file", "rb" ) )

In [ ]:
import tensorflow as tf
import os
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import objectives
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback
from IPython.display import clear_output


# encoder/decoder network size
batch_size=162
original_dim=train_mat.shape[1] # number of jokes
intermediate_dim=int(train_mat.shape[1]*0.6)
latent_dim=int(train_mat.shape[1]*0.2)
nb_epochs=20
epsilon_std=1.0

# encoder network
x=Input(batch_shape=(batch_size,original_dim))
h=Dense(intermediate_dim, activation='tanh')(x)
z_mean=Dense(latent_dim)(h)
z_log_var=Dense(latent_dim)(h)


# sampling from latent dimension for decoder/generative part of network
def sampling(args):
    _mean,_log_var=args
    epsilon=K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=epsilon_std)
    return _mean+K.exp(_log_var/2)*epsilon

z= Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# decoder network
h_decoder=Dense(intermediate_dim, activation='tanh')
x_bar=Dense(original_dim,activation='linear') # this should be linear right?
h_decoded = h_decoder(z)
x_decoded = x_bar(h_decoded)

In [ ]:
x_train=train_mat
x_val = val_mat.todense()

In [ ]:
# Standard VAE recommender


class PlotLosses(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()


# build and compile model
vae = Model(x, x_decoded)
def vae_loss(x,x_bar):
    mask = tf.greater_equal(x, -10) # Masking values corresponding to missing items for each user
    indexes = tf.where(mask)
    reconst_loss=tf.reduce_mean(tf.boolean_mask(tf.pow(x_decoded - x,2), mask))
    #reconst_loss=original_dim*objectives.mse(x,x_bar)
    kl_loss= -0.5*K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return kl_loss + reconst_loss

vae.compile(optimizer='adam', loss=vae_loss)

print("number of training users: ", train_mat.shape[0])


def nn_batch_generator(x, y, batch_size, samples_per_epoch):
    number_of_batches = samples_per_epoch/batch_size
    print(number_of_batches)
    counter=0
    shuffle_index = np.arange(np.shape(y)[0])
    np.random.shuffle(shuffle_index)
    x =  x[shuffle_index, :]
    y =  y[shuffle_index, :]
    while 1:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        x_batch = x[index_batch,:].todense()
        y_batch = y[index_batch,:].todense()
        counter += 1
        yield (np.array(x_batch),np.array(y_batch))
        if (counter >= number_of_batches):
            counter=0


weightsPath = "./weights.hdf5"
checkpointer = ModelCheckpoint(filepath=weightsPath, verbose=1, save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# sending complete training data and shuffle flag will shuffle so that each user comes atleast once in training because of multiple epochs
vae.fit_generator(nn_batch_generator(x_train, x_train, batch_size, 36774), samples_per_epoch=36774, nb_epoch=nb_epochs, 
    validation_data=(x_val, x_val), callbacks=[checkpointer, reduce_lr, plot_losses])

print("training losses over epochs")
print(history.losses)

print("validation losses over epochs")
print(history.val_losses)

In [ ]:
weightsPath = "./weights.hdf5"
vae.load_weights(weightsPath)

In [ ]:
encoder = Model(x, z_mean)
x_train_encoded = encoder.predict(x_train, batch_size=batch_size) 
print(x_train_encoded.shape)

In [ ]:
# Visualizing user embeddings

from sklearn.cluster import KMeans
from bhtsne import tsne

kmeans = KMeans(n_clusters=10, random_state=0).fit(x_train_encoded)
x_train_cluster_labels = kmeans.labels_

x_train_2_embedded = tsne(x_train_encoded.astype('float64'))

plt.figure(figsize=(10, 10))
plt.scatter(x_train_2_embedded[:, 0], x_train_2_embedded[:, 1], c=x_train_cluster_labels)
plt.colorbar()
plt.show()